# Stage LIESSE: Introduction au clustering avec K-means

__Note:__ Ce notebook est inspiré des notebooks 
- https://github.com/Henley13/teaching_td_clustering_2020/blob/main/tp5_clustering_solution.ipynb et 
- https://jonchar.net/notebooks/k-means/,
- ainsi que de l'aide de scikit-learn : https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_assumptions.html#sphx-glr-auto-examples-cluster-plot-kmeans-assumptions-py et https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_assumptions.html#sphx-glr-auto-examples-cluster-plot-kmeans-assumptions-py.

## Chargement des librairies

In [ ]:
%pylab inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')
import pandas as pd

### Définition de K-means

K-means est une méthode de clustering, c'est-à-dire qu'elle a pour but de partitionner les données en différents groupes homogènes, appelés cluster. Contrairement à la classification, le clustering ne dispose pas de labels (mâle/femelle dans les notebooks précédents), mais uniquement des variables.

Il s'agit ainsi d'un méthode d'apprentissage non-supervisé : l'on n'a pas de label pour nous guider, et l'on essaye d'obtenir de l'information sur la "forme" de notre jeu de donnée (le mot *forme* étant compris dans un sens large et à définir).

K-means partitionne notre jeu de donnée $(\boldsymbol{x}_1, ..., \boldsymbol{x}_n)$ en $k$ clusters $(C_1, ..., c_k)$ qui minimisent somme des variance intra-clusters, c'est-à-dire, tels que

$$ \sum_{j = 1}^k \sum_{\boldsymbol{x}_i\in C_j} ||\boldsymbol{x}_i - \boldsymbol{\mu}_j||^2$$

soit minimal, sachant the le *centroide* $\boldsymbol{\mu_j}$ est la moyenne des points appartenant au cluster $C_i$.

K-means est une méthode itérative pour trouver ces clusters. On initialise les centroides et on modifie les clusters de la manière itérative suivante :

1. Assigner chaque point au cluster dont le centroide est le plus proche : $$C_j \gets \{\boldsymbol{x}_i | \forall j' \neq j, ||\boldsymbol{x}_i - \boldsymbol{\mu}_j||^2 \leq ||\boldsymbol{x}_i - \boldsymbol{\mu}_{j'}||^2\}$$


2. Recalculer chaque centroide comme la moyenne des points de son cluster: $$\mu_j \gets \frac{1}{|C_j|}\sum_{\boldsymbol{x}_i \in C_j} \boldsymbol{x}_i$$

L'algorithme s'arrête quand l'assignement des clusters ne change pas d'une itération à l'autre. 

__Quelques remarques sur K-means:__ 
1. L'algorithme dépend des centroides choisis à l'initialisation et peut rester dans un optimum local : il n'y a pas de garantie que k-means trouve les clusters optimaux. En pratique, on peut lancer l'algorithme K-means avec différents centroids initiaux, et choisir un clustering parmi les différents qu'on obtient.
2. K-means associe tous les points les plus proches d'un centroide au cluster équivalent. Par conséquent, les clusters sont définis par le diagramme de Voronoi ([https://fr.wikipedia.org/wiki/Diagramme_de_Vorono%C3%AF](définition) et [https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html#sphx-glr-auto-examples-cluster-plot-kmeans-digits-py](illustration sur scikit-learn)). Implicitement, K-means fait donc l'hypothèse que les clusters convexes, et ne marchera bien que si cette hypothèse est vérifiée.
3. K-means fait aussi l'hypothèse que tous les clusters sont isotropes (c'est-à-dire identiques dans toutes les directions) et ont même variance. [https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_assumptions.html#sphx-glr-auto-examples-cluster-plot-kmeans-assumptions-py](Cet exemple) illustre les problèmes qui interviennes lorsque ces hypothèses ne sont pas vérifiées.

### K-means avec scikit-learn

#### Sur les données réelles `penguins`

Nous allons illuster K-means sur des données réelles.

In [ ]:
penguins = pd.read_csv("data/penguins.csv")

In [ ]:
# Extraire la matrice X et le vecteur y
X = penguins[["bill_length_mm", "bill_depth_mm"]].to_numpy()
y = pd.Categorical(penguins["species"]).astype('category').codes

In [ ]:
from sklearn import cluster

In [ ]:
y_pred = cluster.KMeans(n_clusters=3, random_state=34).fit_predict(X)

On peut représenter les clusters appris par K-means:

In [ ]:
plt.figure(figsize = (8, 4))

plt.subplot(121)
for label in [0, 1, 2]:
    plt.scatter(X[y_pred == label, 0], X[y_pred == label, 1], label=label)
plt.title("Clustering avec K-means (k = 3)")
plt.xlabel("Longueur du bec (mm)")
plt.ylabel("Hauteur du bec (mm)")
plt.legend()

plt.subplot(122)
for i, label in enumerate(list(set(penguins.loc[:, "species"]))):
    data_label = X[penguins.loc[:, "species"] == label, :]
    plt.scatter(data_label[:, 0], data_label[:, 1],
                s=30, label=label)
plt.title("Espèces de manchot")
plt.xlabel("Longueur du bec (mm)")
plt.ylabel("Hauteur du bec (mm)")
plt.legend()
plt.tight_layout()

K-means ne parvient pas à choisir des clusters qui correspondent aux trois espèces.

#### Sur données simulées

Nous allons définir un jeu de données simulées que nous utiliserons pour le reste du notebook.

In [ ]:
from sklearn import datasets

Nous générons des données à partir d'un mélange de trois distributions gaussiennes:

In [ ]:
n_samples = 1500
X, label = datasets.make_blobs(n_samples=n_samples, 
                                  cluster_std=[1.0, 2.5, 0.5], 
                                  random_state=170)

On représente ces données:

In [ ]:
df_blobs = pd.DataFrame({"x1": X[:, 0],
                         "x2": X[:, 1],
                         "label": label})


In [ ]:
plt.figure(figsize=(5, 5))
for i in [0, 1, 2]:
    plt.scatter(X[label == i, 0], X[label == i, 1], label = i)
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.title("Labels utilisés pour générer les données")
plt.legend(prop={'size': 15})

In [ ]:
clusters = cluster.KMeans(n_clusters = 3, random_state = 45).fit_predict(X)

In [ ]:
plt.figure(figsize=(5, 5))
for i in [0, 1, 2]:
    plt.scatter(X[clusters == i, 0], X[clusters == i, 1], label = i)
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.title("Cluster obtenus par K-Means")
plt.legend(title = "clusters")

Nous obtenons des clusters assez proches des vrais clusters. Peut-on mesurer la qualité de ce clustering?

### Mesurer la similarité entre deux clusterings

In [ ]:
from sklearn import metrics

Nous voulons mesurer à quel point le clustering obtenu par k-means est proche des vrais classes. Pour ce faire, nous utilisons le l'index de Rand. 
Considérons deux partitionnements des données $\mathcal{P}_1$ $\mathcal{P}_2$, qui correspondent à deus clustering. Définissons le quantités
- $a$: nombre de paires de points qui sont dans le même cluster pour $\mathcal{P}_1$ et pour $\mathcal{P}_2$.
- $b$: nombre de paires de points qui sont dans des clusters différents pour $\mathcal{P}_1$ et pour $\mathcal{P}_2$.
- $c$: nombre de paires de points qui sont dans le même cluster pour $\mathcal{P}_1$ et pour des clusters différents pour $\mathcal{P}_2$.
- $d$: nombre de paires de points qui sont dans des clusters différents pour $\mathcal{P}_1$ et dans le même cluster pour $\mathcal{P}_2$.
On voit que $a + b$ est le nombre de pairs consistantes entre les deux partitions et $c + d$ représente le nombre de pairs inconsistantes entre les deux partitions.

Nous définissons alors
$$R = \frac{a+b}{a+b+c+d} = \frac{a+b}{n\choose{2}}.$$

L'index de Rand est compris entre $0$ et $1$ et permet de mesurer la consistance entre deux clusterings. En pratique, nous comparerons le clustering utilisé par K-means aux vrais labels utilisées pour générer les données.

Mais il ne s'agit pas d'apprentissage supervisé ! puisque ces labels n'ont pas été utilisés pour entraîner la méthode de clustering. Nous pouvons utiliser l'indice de Rand parce que nous savons qu'il y a trois vraies classes qui ont permis de générer les données. Mais dans les applications pratiques, le clustering n'est pas utilisé pour trouver des classes qui correspondent à des labels dont l'on disposerait (sinon, on utiliserait de la classification supervisée).

In [ ]:
metrics.rand_score(clusters, label)

C'est un score élevé. L'index de Rand est plus faible pour le clustering des données penguins :

In [ ]:
print("L'index de Rand des clusters de manchots est %.2f." % metrics.rand_score(y, y_pred))

On peut aussi utiliser l'index de Rand ajusté ([https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html](réference)), qui compense le fait que deux clusterings choisis aléatoirements n'ont pas un index de Rand nul.

In [ ]:
print("L'index de Rand ajusté des clusters de manchots est %.2f." % metrics.adjusted_rand_score(y, y_pred))

### Choisir le nombre de cluster

Nous allons utiliser un critère pour choisir le bon nombre de cluster. Nous allons utiliser le coefficient de silhouette ([https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html](référence))).

Soit $a(\boldsymbol{x})$ la distance moyenne intra-cluster pour le point $\boldsymbol{x}$:
$$a(\boldsymbol{x}) = \frac{1}{|C_k| - 1} \sum_{\boldsymbol{x'}\in C_k \backslash \boldsymbol{x}} ||\boldsymbol{x} - \boldsymbol{x'}||.$$ 
Soit $b(\boldsymbol{x})$ la distance moyenne au plus proche cluster:
$$b(\boldsymbol{x}) = \min_{l\neq k} \frac{1}{|C_k| - 1} \sum_{\boldsymbol{x'} \in C_l} ||\boldsymbol{x} - \boldsymbol{x'}||.$$ 
$a(\boldsymbol{x})$ quantifie à quel point $\boldsymbol{x}$ est bien intégré à son cluster et $b(\boldsymbol{x})$ quantifie à quel point $\boldsymbol{x}$ serait bien intégré à un autre cluster.

Le coefficient de silhouette est défini par:
$$s(\boldsymbol{x}) = \frac{b(\boldsymbol{x}) - a(\boldsymbol{x})}{\max(a(\boldsymbol{x}), b(\boldsymbol{x})}.$$
Il est compris entre $-1$ (si $\boldsymbol{x}$ est très proche des points d'un autre cluster) et $1$ (si $\boldsymbol{x}$ est très proche des points de son cluster).

La somme $s = \sum_{\boldsymbol{x}} s(\boldsymbol{x})$ des scores de silhouette donne un score global de la "qualité" du clustering.

Nous pouvons maintenant utiliser ce score pour comparer les clusterings obtenus avec K-means pour différents nombres de clusters:

In [ ]:
y_pred2 = cluster.KMeans(n_clusters=2, random_state=34).fit_predict(X)
y_pred3 = cluster.KMeans(n_clusters=3, random_state=34).fit_predict(X)
y_pred4 = cluster.KMeans(n_clusters=4, random_state=34).fit_predict(X)
y_pred5 = cluster.KMeans(n_clusters=5, random_state=34).fit_predict(X)

In [ ]:
plt.figure(figsize = (10, 10))
plt.subplot(221)
plt.scatter(X[:, 0], X[:, 1], 
            c=y_pred2)
plt.title("k = 2, silhouette_score = %.2f" % \
          metrics.silhouette_score(X, y_pred2))
plt.xlabel("Longueur du bec (mm)")
plt.ylabel("Hauteur du bec (mm)")
plt.subplot(222)
plt.scatter(X[:, 0], X[:, 1], 
            c=y_pred3)
plt.title("k = 2, silhouette_score = %.2f" % \
          metrics.silhouette_score(X, y_pred3))
plt.xlabel("Longueur du bec (mm)")
plt.ylabel("Hauteur du bec (mm)")

plt.subplot(223)
plt.scatter(X[:, 0], X[:, 1], 
            c=y_pred4)
plt.title("k = 2, silhouette_score = %.2f" % \
          metrics.silhouette_score(X, y_pred4))
plt.xlabel("Longueur du bec (mm)")
plt.ylabel("Hauteur du bec (mm)")

plt.subplot(224)
plt.scatter(X[:, 0], X[:, 1], 
            c=y_pred5)
plt.title("k = 2, silhouette_score = %.2f" % \
          metrics.silhouette_score(X, y_pred5))
plt.xlabel("Longueur du bec (mm)")
plt.ylabel("Hauteur du bec (mm)")


En utilisant la métrique du score de silhouette, **il est préférable de choisir 3 clusters**. C'est la *bonne* réponse, puisqu'on a utilité 3 gaussiennes pour générer les données.

Pour aller plus loin, on peut vouloir représenter la *distribution* des silhouettes de chaque cluster, au lieu d'uniquement calculer leurs moyennes. Nous utilisons pour cela la fonction `silhouette_samples` :

In [ ]:
# get input values
X = df_blobs.loc[:, ["x1", "x2"]].values

# one silhouette plot for different numbers of clusters.
for n_clusters in [2, 3, 4, 9]:
    
    # create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(10, 4)
    
    # demarcate silhouette plots of individual clusters by inserting blanck space
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # K-means clustering
    kmean = cluster.KMeans(
        n_clusters=n_clusters, init='k-means++', 
        n_init=10, max_iter=300, tol=0.0001, 
        verbose=0, random_state=13, 
        algorithm='auto')
    cluster_labels = kmean.fit_predict(X)

    # compute average silhouette score
    score = metrics.silhouette_score(X, cluster_labels)

    # compute silhouette scores for each sample
    sample_silhouette_values = metrics.silhouette_samples(X, cluster_labels)

    # plot silhouette scores for each sample
    y_lower = 10
    for i in range(n_clusters):
        
        # aggregate the silhouette scores for samples belonging to cluster i and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    # axis labels and title
    ax1.set_title("Average silhouette coefficient: {0:0.3f}".format(score), fontsize=10)
    ax1.set_xlabel("Silhouette coefficient values", fontsize=15)
    ax1.set_ylabel("Cluster label", fontsize=15)

    # draw a vertical line for average silhouette score of all the values
    ax1.axvline(x=score, color="red", linestyle="--")

    # clear the yaxis labels / ticks
    ax1.set_yticks([])  
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X[:, 0], X[:, 1], 
                marker='o', s=30, alpha=0.8, c=colors, edgecolor='k')    
    
    # draw white circles at cluster centers
    centers = kmean.cluster_centers_
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')
        
    # axis labels and title
    ax2.set_title("Number of clusters: {0}".format(n_clusters), fontsize=15)
    ax2.set_xlabel("X1", fontsize=10)
    ax2.set_ylabel("X2", fontsize=10)

    # main title
    plt.suptitle(("Silhouette analysis for K-means clustering on sample data "
                  "with n_clusters = {0}".format(n_clusters)),
                 fontsize=10, fontweight='bold')

plt.show()

Nous voyons que la distribution des score de silhouette dans chaque cluster représente bien à quel point le cluster est "clairement identifié" comparé aux autre clusters. 

Lorsque beaucoup de points dans un cluster a un coefficient de silhouette élevé, la distribution de coefficients est concentrée autour d'une valeur élevée, comme c'est le cas avec $k = 2$ et $k = 3$ clusters (correspondant aux figures des deux lignes du haut).

En revanche, lorsqu'un clusters a beaucoup de coefficients de silhouette autour de valeurs faible, le discribution des coefficients est presque uniforme entre $0$ et une valeur faiblement élevée ($\sim 0.5$), comme c'est le cas avec $k=4$ et $k = 9$ clusters (correspondant aux figures des deux lignes du bas).

## Implémentation de K-means en python

Quelques fonctions utiles:

In [ ]:
# Initialiser les clusters aléatoirement
def initialize_clusters(points, k, random_state = None):
    """Initializes clusters as k randomly selected points from points."""
    if random_state is not None:
        numpy.random.seed(random_state)
    return points[np.random.randint(points.shape[0], size=k)]
    
# Calcul des distances entre points de centroides
def get_distances(centroid, points):
    """Returns the distance the centroid is from each data point in points."""
    return np.linalg.norm(points - centroid, axis=1)

Implémentation:

In [ ]:
k = 3
max_iter = 50

# Initialize our centroids by picking random data points
centroids = initialize_clusters(X, k, random_state=17)

# Initialize the vectors in which we will store the
# assigned classes of each data point and the
# calculated distances from each centroid
classes = np.zeros(X.shape[0], dtype=np.float64)
distances = np.zeros([X.shape[0], k], dtype=np.float64)

## Define oject to keep the kmeans iterations results in memory
classes_all = np.zeros((X.shape[0], max_iter), dtype = np.int16)
centroids_all = np.zeros((centroids.shape[0], centroids.shape[1], max_iter))

# Loop for the maximum number of iterations
for i in range(max_iter):
    
    # Assign all points to the nearest centroid
    for j, c in enumerate(centroids):
        distances[:, j] = get_distances(c, X)
        
    # Determine class membership of each point
    # by picking the closest centroid
    classes = np.argmin(distances, axis=1)
    classes_all[:, i] = classes
    
    # Update centroid location using the newly
    # assigned data point classes
    for c in range(k):
        centroids[c] = np.mean(X[classes == c], 0)
    centroids_all[:, :, i] = centroids

Nous pouvons afficher le résultat:

In [ ]:
for label in [0, 1, 2]:
    plt.scatter(X[classes == label,0], X[classes == label, 1], alpha=0.5)
plt.scatter(centroids[:,0], centroids[:,1], color=['blue', 'darkred', 'green'], marker='o', lw=2)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title("Clustering obtained by scikit-learn")

Nous pouvons aussi représenter les différentes étapes de K-means:

In [ ]:
plt.figure(figsize = (8, 8))
for ind in range(0, 6):
    plt.subplot(3, 2, ind + 1)
    for label in [0, 1, 2]:
        plt.scatter(X[classes_all[:, ind] == label,0], X[classes_all[:, ind] == label, 1], alpha=0.5)
    
    plt.scatter(centroids_all[:,0 , ind], centroids_all[:, 1, ind],\
                color=['blue', 'darkred', 'green'], marker='o', lw=2)
    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")
    plt.tight_layout()
    plt.title("Iteration #" + str(ind))

## Pour aller plus loin

1. Sur le jeu de données `penguins` K-means ne parvenait pas à séparer les trois espèces. 

- Utilisez l'analyse de score de silhouette pour vérifier que k = 3 est le nombre "optimal" de clusters
- Utilisez K-means en ajoutant la variable "flipper_length_mm" aux deux variables "bill_length_mm" et "bill_depth_mm". Utilisez l'index de Rand pour estimer si le clustering obtenu est plus proche des trois espèces de manchots.

2. D'autres méthodes de clustering sont présentées [https://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods](ici).

- En observant les résultats des différentes méthodes sur le jeu de données à la première rangée, quelles méthodes semblent ne pas faire l'hypothèse que les clusters sont convexes ?
- En observant les résultats des différentes méthodes sur la quatrième rangée, quelles méthodes semblent ne pas faire l'hypothèse que les clusters sont isotropes ?